# Runs for Micro-wrapper development

In [1]:
import black
import jupyter_black

jupyter_black.load(
    lab=True,
    line_length=110,
    target_version=black.TargetVersion.PY311,
)

In [2]:
import os
import subprocess
import sys

sys.path.insert(1, os.path.abspath(".."))

import GooseSLURM as gs
import numpy as np

import lysis

In [3]:
micro_scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("nodes", int),
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("diss_const_tPA_wPLG", float),
        ("diss_const_tPA_woPLG", float),
    ]
)
macro_scenario_type = np.dtype(
    [
        ("descriptor", np.str_, 40),
        ("total_molecules", int),
        ("pore_size", float),  # in microns. Code requires cm
        ("fiber_diameter", float),  # in microns. Needs to be added to pore_size to get grid_node_distance
        ("binding_sites", int),
        ("cols", int),
        ("rows", int),
        ("empty_rows", int),
        ("forced_unbind", float),
        ("average_bind_time", float),
    ]
)
mechanism_type = np.dtype([("descriptor", np.str_, 40), ("executable", np.str_, 40)])
run_type = np.dtype(
    [
        ("exp_code", np.str_, 15),
        ("source_code", np.str_, 15),
        ("macro_scenario", np.str_, 40),
        ("micro_scenario", np.str_, 40),
        ("mechanism", np.str_, 40),
        ("seed", np.int64),
    ]
)
diameter_code = {
    46: "Q0",
    72.7: "Q2",
    105.1: "TF",
    123.0: "TB",
    145.4: "Q4",
}

In [4]:
micro_scenarios = np.array(
    [
        ("0.1-PKd", 7,  72.7, 0.002, 0.036),
        ("1-PKd",   7,  72.7, 0.02,  0.36),
        ("10-PKd",  7,  72.7, 0.2,   3.6),
        ("TN-L",    7,  72.7, 0.02,  0.36),
        ("TN-D",    7,  72.7, 0.02,  0.36),
        ("TK-L",   13, 145.4, 0.02,  0.36),
        ("TK-D",   13, 145.4, 0.02,  0.36),
        ("TB-xi",  11, 123,   0.02,  0.36),
        ("TF-x",   10, 105.1, 0.02,  0.36),
        ("TF-vii",  7, 105.1, 0.02,  0.36),
        ("TF-v",    5, 105.1, 0.02,  0.36),
    ],
    dtype=micro_scenario_type,
)
macro_scenarios = np.array(
    [
        ("0.1-PKd",        43_074, 1.0135,  72.7, 427,     93, 121,  28, 0.5143,   277.8),
        ("1-PKd",          43_074, 1.0135,  72.7, 427,     93, 121,  28, 8.52e-2,   27.8),
        ("10-PKd",         43_074, 1.0135,  72.7, 427,     93, 121,  28, 5.4e-3,     2.78),
        ("TN-L__9_350",     9_350, 1.0135,  72.7, 427,     93,  93,   0, 0.0852,    27.8),
        ("TN-D__9_350",     9_350, 0.22,    72.7, 427,    342, 116,   0, 0.0852,    27.8),
        ("TK-L__9_350",     9_350, 1.0135, 145.4, 213,     87,  29,   0, 0.0729129, 27.8),
        ("TK-D__9_350",     9_350, 0.22,   145.4, 213,    274,  42,   0, 0.0729129, 27.8),
        ("TB-xi__21_105",  21_105, 5.34,   123,   248.531, 19, 184,  83, 0.077725,  27.8),
        ("TF-x__9_951",     9_951, 2.5,    105.1, 306.357, 39, 385, 326, 0.078644,  27.8),
        ("TF-vii__9_951",   9_951, 2.5,    105.1, 204.238, 39, 385, 265, 0.086698,  27.8),
        ("TF-v__9_951",     9_951, 2.5,    105.1, 136.159, 39, 385, 151, 0.084703,  27.8),
    ],
    dtype=macro_scenario_type,
)
mechanisms = np.array(
    [
        ("Into and along - External", "macro_diffuse_into_and_along__external"),
        ("Into and along - Internal", "macro_diffuse_into_and_along__internal"),
    ],
    dtype=mechanism_type,
)

In [5]:
in_file_code = "_PLG2_tPA01_{input_code}.dat"
out_file_code = "_{scenario_code}"

In [6]:
# seed sequence entropy: 234567848524899077628396107049383963466
group_code = "2024-09-02-14"
runs = np.array(
    [
        (group_code + "00", "2023-02-02-2203", "Phys-Kd", "Into and along - External", -2137354075),
        (group_code + "01", "2023-02-02-2208", "10x-sm-Kd", "Into and along - External", -854989241),
        (group_code + "02", "2023-02-02-2213", "10x-bg-Kd", "Into and along - External", -1212172957),
        (group_code + "03", "2023-12-10-1900", "TN-L_9350", "Into and along - Internal", 1_034_836_197),
        (group_code + "04", "2023-12-10-1903", "TN-D_9350", "Into and along - Internal", 129_314_213),
        (group_code + "05", "2023-12-10-1906", "TK-L_9350", "Into and along - Internal", 2_608_028_918),
        (group_code + "06", "2023-12-10-1909", "TK-D_9350", "Into and along - Internal", 106_672_317),
        (group_code + "07", "2024-04-16-1912", "TB-xi__21_105", "Into and along - External", 2_061_649_767),
        (group_code + "08", "2024-04-16-1913", "TF-x__9_951", "Into and along - External", 3_687_972_975),
        (group_code + "09", "2024-04-16-1914", "TF-vii__9_951", "Into and along - External", 1_422_833_478),
        (group_code + "10", "2024-04-16-1915", "TF-v__9_951", "Into and along - External", 77_748_149),
    ],
    dtype=run_type,
)

In [7]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    p = {
        "total_molecules": int(scen["total_molecules"]),
        "pore_size": scen["pore_size"] / 10_000,  # convert to cm
        "grid_node_distance": scen["fiber_diameter"] / 1_000 + scen["pore_size"],
        "binding_sites": int(scen["binding_sites"]),
        "cols": int(scen["cols"]),
        "rows": int(scen["rows"]),
        "empty_rows": int(scen["empty_rows"]),
        "forced_unbind": scen["forced_unbind"],
        "average_bind_time": scen["average_bind_time"],
        "seed": int(run["seed"]),
        "macro_version": mech["executable"],
    }
    e.initialize_macro_param(p)
    e.to_file()

    with open(os.path.join(e.os_path, "README.md"), "w") as file:
        file.write(
            f""" ## {e.experiment_code}
Lysis Front study
Array version
New scenarios from bench experiments

Uses Fortran Macroscale code with
    -  Diffuse into and along
    -  'restricted move' bug correction
    -  'passerby molecule' bug correction
    -  addition of extra output data
    -  changing the termination criteria from 'fixed time' to 'all fibers degraded'
    -  changing macro-unbind wait time to remaining leaving time.
    -  eliminating unused data variables
    -  Moved degraded fiber check into molecule loop
    -  Removed "degrade" array and use "t_degrade" instead
    -  Read in "neighborc" array generated in Python
    -  Output molecule bind/unbind times for transit time calculations
    -  Output molecule bind/unbind locations.

This set of data is from the "{run['scenario']}" set.
"""
        )

    result = subprocess.run(
        ["cp", os.path.join("src", "fortran", mech["executable"] + ".f90"), e.os_path],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1912'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1913'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1914'], returncode=0, stdout=b'', stderr=b'')
CompletedProcess(args=['cp', 'src/fortran/macro_diffuse_into_and_along__external.f90', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1915'], returncode=0, stdout=b'', stderr=b'')


In [8]:
folder_vars = """
homedir="/home/bpaynter/git/UCO-OpResearch/lysis"
workdir="/tmp/bpaynter/${SLURM_ARRAY_JOB_ID}-${SLURM_ARRAY_TASK_ID}"
datadir="${workdir}/data/${exp_code}"
"""

transfer_script = """
# 1. Transfer to node 
# ====================
echo "Starting data transfer to node."

# create/empty the temporary directory on the compute node
if [ ! -d "${workdir}" ]; then
  mkdir -p "${workdir}"
else
  echo rm -ri "${workdir}"/*
fi

mkdir -p "${datadir}"

cp ${homedir}/data/${exp_code}/*${in_code} ${datadir}/
cp ${homedir}/data/${exp_code}/params.json ${datadir}/
cp ${homedir}/bin/${fort_executable} ${workdir}/
echo "Data transfer to node completed."
"""

return_script = """
# 2. Function to transfer back to the head node 
# ==============================================

# define clean-up function
function clean_up {
  sim=$(printf "%02d" ${SLURM_ARRAY_TASK_ID})
  # - copy everything from the temporary directory on the compute-node
  echo "Copying data back to home directory."
  mkdir -p ${homedir}/data/${exp_code}/$sim
  cp -prf "${datadir}"/*${out_code}* ${homedir}/data/${exp_code}/$sim/
  cp -prf "${datadir}"/*.txt ${homedir}/data/${exp_code}/$sim/
  # - erase the temporary directory from the compute-node
  echo "Erasing temporary directories."
  rm -rf "${workdir}"/*
  rm -rf "${workdir}"
  # - exit the script
  echo "Job complete. Exiting."
  exit
}

# call "clean_up" function when this script exits, it is run even if SLURM cancels the job
trap 'clean_up' EXIT
"""
execute_script = """
# 3. Execute 
# ===========

source /home/bpaynter/.bashrc
source /home/bpaynter/lysis.sh
cd ${homedir}/src/python
echo "Starting code execution."

python -u fortran_run.py \
    --in_code ${in_code} \
    --out_code ${out_code}.dat \
    -n $SLURM_ARRAY_TASK_ID \
    --cwd ${workdir} \
    ${workdir}/${fort_executable} \
    ${exp_code}
    
sleep 10
echo "Code execution complete."
"""

In [9]:
for run in runs:
    mech = mechanisms[mechanisms["descriptor"] == run["mechanism"]][0]
    scen = scenarios[scenarios["descriptor"] == run["scenario"]][0]
    e = lysis.util.Experiment(
        os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis", "data"),
        experiment_code=run["exp_code"],
    )
    e.read_file()
    run_vars = f"""
exp_code="{e.experiment_code}"
in_code="{in_file_code.format(input_code=run["scenario"].split("_")[0])}"
out_code="{out_file_code.format(scenario_code=run['scenario'])}"
fort_executable="{mech['executable']}"
echo "Experiment ${{exp_code}} (${{out_code}}), Simulation #${{SLURM_ARRAY_TASK_ID}}"
"""
    # job-options
    sbatch = {
        "job-name": f"lysis-{e.experiment_code}",
        "out": os.path.join(e.os_path, "job.slurm-%A-%a.out"),
        "array": f"0-{e.macro_params.total_trials-1}",
        "nodes": 1,
        "mem": 3096,
        "ntasks": 1,
        "cpus-per-task": 1,
        "exclusive=user": "",
        # "exclusive": "",
        "partition": "long",
    }
    script = [run_vars, folder_vars, transfer_script, return_script, execute_script]
    with open(os.path.join(e.os_path, "job.slurm"), "w") as file:
        file.write(gs.scripts.plain(script, **sbatch))
    result = subprocess.run(
        ["sbatch", os.path.join(e.os_path, "job.slurm")],
        cwd=os.path.join("/", "home", "bpaynter", "git", "UCO-OpResearch", "lysis"),
        capture_output=True,
    )
    print(result)

CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1912/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223365\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1913/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223366\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1914/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223367\n', stderr=b'')
CompletedProcess(args=['sbatch', '/home/bpaynter/git/UCO-OpResearch/lysis/data/2024-04-16-1915/job.slurm'], returncode=0, stdout=b'Submitted batch job 2223368\n', stderr=b'')
